# Optimizando el Diseño de la Ciclovía de la Ciudad de México con un Algoritmo tipo Metropolis-MonteCarlo 

## El análogo al Modelo de Ising

El algoritmo de Metropolis es ampliamente aplicado en simulaciones computacionales del modelo de Ising, en el que se estudia un sistema físico que consta de partículas dispuestas en un *lattice* cuyas interacciones entre sí mismas y con un campo externo determinan su estado (que toma los valores de +1 o -1) y por ende el estado global del sistema dada una temperatura.

Mi objetivo en este trabajo es proponer un algoritmo de Metropolis que permita optimizar el diseño de las ciclovías de la Ciudad de México dado un presupuesto, un campo de necesidades y los diferentes estados que puede tomar cada calle dentro de una red, analizando cada par de circuitos conectados por separado. 

### Los estados y la *lattice*

***
#### Los estados posibles por calle
Primero es necesario crear el objeto computacional con el que voy a tratar, que es una red de calles con distintos estados posibles. Los estados que voy a considerar son

| $  | ID  | cap | att | características del carril                  |
|----|-----|-----|-----|---------------------------------------------|
| 13 | CTR | 4   | 7   | calle cetram                                |
| 12 | ESS | 3   | 6   | exclusivo con ecobici y biciestacionamiento |
| 11 | ESN | 3   | 5   | exclusivo con ecobici                       |
| 10 | ENS | 3   | 4   | exclusivo con biciestacionamiento           |
| 19 | ENN | 3   | 3   | exclusivo sin estaciones                    |
| 8  | BSS | 2   | 5   | bus-bici con ecobici y biciestacionamiento  |
| 7  | BSN | 2   | 4   | bus-bici con ecobici                        |
| 6  | BNS | 2   | 3   | bus-bici con biciestacionamiento            |
| 5  | BNN | 2   | 2   | bus-bici sin estaciones                     |
| 4  | MSS | 1   | 4   | mixto con ecobici y biciestacionamiento     |
| 3  | MSN | 1   | 3   | mixto con ecobici                           |
| 2  | MNS | 1   | 2   | mixto con biciestacionamiento               |
| 1  | MNN | 1   | 1   | mixto sin estaciones                        |
| 0  | NNN | 0   | 0   | sin ciclovía                                |

Entonces, considero la siguiente definición
\begin{definition}
El vector de estado $\vec{S}$ de una calle se define como
$$\vec{S} = (s,\$, \text{id},c_s,a_s)$$

donde: $s$ es el valor de spin (el que se utilizará en el término de interacción entre vecinos del hamiltoniano) del estado, $ \$ $ es el costo por metro lineal del estado, $\text{id}$ es el identificador, $c_s$ es la capacidad de uso del estado, $a_s$ es la capacidad de atracción y conversión de usuarios del estado.
\end{definition}

En la siguiente celda defino el objeto computacional necesario.

In [1]:
struct streetState
    # estos valores son fijos
    id::String # identificador de un estado
    # estos valores se utilizarán para calcular las interacciones entre estados y con las necesidades
    spin::Int # el spin del estado
    cost::Float64 # el costo por metro lineal del estado
    cap::Int # la capacidad de diseño del estado
    attPot::Int # la capacidad de acceso a usuarios del estado
    # estos valores se calculan
    intField::Float64 # el valor de interacción del estado con el campo
    totCost::Float64 # el valor de costo total del estado dada su existencia en la red
end

list_states = [[-1,0,"NNN",0,0],
               [1,1,"MNN",1,1],
               [1,2,"MNS",1,2],
               [1,3,"MSN",1,3],
               [1,4,"MSS",1,4],
               [1,5,"BNN",2,2],
               [1,6,"BNS",2,3],
               [1,7,"BSN",2,4],
               [1,8,"BSS",2,5],
               [1,9,"ENN",3,3],
               [1,10,"ENS",3,4],
               [1,11,"ESN",3,5],
               [1,12,"ESS",3,6],
               [1,13,"MSS",4,7]]

function setstate!(s::streetState,x::Int) 
    index = x+1
    s.spin = list_states[index][1]
    s.cost = x
    s.id = list_states[index][3]
    s.cap = list_states[index][3]
    s.attPot = list_states[index][4]
end

setTotCost!(s::streetState, x::Float64) = s.totCost = x
setifl!(s::streetState, x::Float64) = s.intField = x

setifl! (generic function with 1 method)

***
#### La *lattice*: la red de calles
La red de calles será la que esté delimitada por las regiones geográficas que ocupen los dos distritos en cuestión, se requieren las coordenadas de cada una de las calles de la red, sus características y la lista de vecinos. En la siguiente celda defino el objeto correspondiente a la red de calles. Las características deben ser los atributos necesarios que permitan calcular las interacciones con la red de ciclovías que es mi objeto de diseño.

In [3]:
struct streetNetwork
    # estos valores se definen el general, los primeros no se modifican
    coords::Vector{Vector{NTuple{2,Float64}}} # la lista de coordenadas de las esquinas de cada calle de la red
    longs::Vector{Float64} # la lista de longitudes reales de cada calle en la red
    neighbors::Vector{Vector{Int}} # la lista de vecinos de cada calle de la red
    #= este puede modificarse con los datos de Mariana, en principio son densidades de población, existencia de estaciones
    cercanas de metro, metrobús, etc =#
    usrDnst::Vector{Float64} # la densidad de potenciales usuarios de la red por cada calle (se puede trabajar con AGEBs)
    # estos sí se modifican
    states::Vector{streetState} # la lista de estados de las calles de la red
    locField::Vector{Vector{Float64}} # la lista con los vectores del campo para cada elemento de la red
    totBikewaysCost::Float64 # el costo total de la ciclovía en la red
    # a partir de aquí tienen que ver con el par de distritos a estudiar, ninguno se modifica
    dstrCoords::Vector{NTuple{2,Float64}} # las coordenadas de los puntos notables de los distritos (pueden ser centroides)
    demand::Float64 # la demanda de usuarios por conexión, que se obtiene con el otro algoritmo
end

setlocFieldi!(sn::streetNetwork,i::Int,f::Vector{Float64}) = sn.locField[i] = f
setBikewaysCost!(sn::streetNetwork,c::Float64) = s.totBikewaysCost = c
setstatei!(sn::streetNetwork,i::Int,s::streetState) = sn.states[i] = s

setlocFieldi! (generic function with 1 method)

### El  campo y las interacciones con los estados

*** 
#### El campo local inicial
En principio, propongo un campo con 4 entradas asociadas con las necesidades de ciclovía, el impacto al tránsito y la exposición a la contaminación, esto es

\begin{definition}
El vector de campo local $\vec{B_i}$ se define como
$$\vec{B_i} = (d \times u_n,d \times p_n,t,e) = (u_{wn},p_{wn},t,e)$$

donde, para la $i$-ésima calle: $u_n$ es la demanda de usuarios (por enlace entre pares), $p_n$ es la densidad de potenciales usuarios (por región), $t$ es el impacto al tránsito que tiene el estado de la calle, $e$ es la exposición a la contaminación de la calle. Los dos primeros términos están pesados por una distribución normal $d$ centrada en la ruta ideal entre (el vector que une) cada par de distritos.
\end{definition}

La multiplicación de los primeros términos por la distribución normal garantiza que sea mayor el campo (la necesidad de poner ciclovía) en las calles cercanas a la ruta ideal. En la siguiente celda defino las funciones que realizarán los cálculos básicos para $d$, $l_f$ y los cálculos iniciales del campo local de toda la red.

In [4]:
function d(sn::streetNetwork,i::Int)
    #= falta condicionar las coordenadas =#
    x1 = sn.dstrCoords[1][1]
    y1 = sn.dstrCoords[1][2]
    x2 = sn.dstrCoords[2][1]
    y2 = sn.dstrCoords[2][2]
    x01 = sn.coords[i][1][1]
    y01 = sn.coords[i][1][2]
    x02 = sn.coords[i][2][1]
    y02 = sn.coords[i][2][2]
    # calculando la distancia entre las esquinas de la i-ésima calle y la línea que une al p-ésimo par de distritos
    ydiff = y2-y1
    xdiff = x2-x1
    xyp = x2*y1-x1*y2
    sqxy = sqrt(ydiff^2 + xdiff^2)
    r1 = abs(ydiff*x01-xdiff*y01+xyp)/sqxy
    r2 = abs(ydiff*x02-xdiff*y02+xyp)/sqxy
    #= calculando el valor de la distribución, el valor de sigma debe calcularse (se me ocurre como función de las longitudes o de la 
    dispersión de densidades de población y empleo o ambas) =#
    σ = 1
    d1 = (1/(2*π*σ^2))*exp(r1^2/(2/σ^2))
    d2 = (1/(2*π*σ^2))*exp(r1^2/(2/σ^2))
    d = (d1+d2)/2
    return d
end

function localField(sn::streetNetwork,i::Int)
    w = d(s,i)
    uwn = w*s.demand
    pwn = w*s.usrDnst[i] 
    # estos valores están determinados por los resultados de Mariana y de Abraham
    t = 0
    e = 0
    f = [uwn,pwn,t,e]
    setlocFieldi!(sn,i,f)
end

function Btot(sn::streetNetwork)
    for i in 1:length(sn.longs)
        localField(sn,i)
    end
end

B (generic function with 1 method)

***
#### Las interacciones de un estado con el campo
Para definir las interacciones entre el estado de una calle y el campo voy a considerar la diferencia entre las características de diseño y lo que demanda la sociedad. Entonces, defino la operación producto como

\begin{definition}
La operación $\langle\vec{B_i},\vec{S_i}\rangle$ para la interacción estado-campo de la $i$-ésima calle se define como
$$\langle\vec{B_i},\vec{S_i}\rangle = ||(u_{wn}-c_s,p_{wn}-a_s,t,e)||= \sqrt{(u_{wn}-c_s)^2+(p_{wn}-a_s)^2+t^2+e^2}$$
\end{definition}

En la siguiente celda defino la función de interacción

In [7]:
function interBiSi(sn::streetNetwork,i::Int)
    fieldi = sn.locField[i]
    statei = sn.states[i]
    uwn = fieldi[1]
    pwn = fieldi[2]
    t = fieldi[3]
    e = fieldi[4]
    cs = statei.cap
    as = statei.attPot
    inter = sqrt((uwn-cs)^2+(pwn-as)^2+t^2+e^2)
    return inter
end

interBiSi (generic function with 1 method)

### El Hamiltoniano

El hamiltoniano que representa la energía a minimizar es entonces
\begin{definition}
El hamiltoniano del sistema
$$ H = -J \sum_{<i,j>}^{N} \sigma_i \sigma_j + \sum_i^N \langle\vec{B_i},\vec{S_i}\rangle + \sum_{\sigma_i=1}^N l_i \$\_i $$

donde:
- el primer término controla la conectividad (es mínimo cuando existe toda la ciclovía o cuando no existe ninguna en la red)
- el segundo término controla la interacción de los estados de las calles con el campo (es mínimo cuando los estados interactúan mejor con el campo local)
- el tercero controla el costo de la ciclovía (es mínimo cuando el costo es el menor)
\end{definition}

Hay que notar que el segundo y el tercer término son siempre positivos y actúan como penalizaciones a la minimización de la energía. En la siguiente celda defino la función que calcula el costo total de un estado y el hamiltoniano de la red.

In [10]:
function costi(sn::streetNetwork,i::Int)
    statei = sn.states[i]
    longi = sn.longs[i]
    costi = statei.cost
    totCost = longi*costi
    return totCost
end

J = 1
function Htot(sn::streetNetwork)
    interList = []
    costList = []
    interTerm = 0
    for i in 1:length(sn.states)
        statei = sn.states[i]
        cst = costi(sn,i)
        itr = interbisi(sn,i)
        setTotCost!(statei,cst) 
        setifl!(statei,itr)
        push!(costList,cst)
        push!(interList,itr)
        for j in sn.neighbors[i]
            statej = sn.states[j]
            interTerm += (J/2)*statei.spin*statej.spin
        end
    end
    fieldTerm = sum(interList)
    costTerm = sum(costList)
    setBikewaysCost!(sn,costTerm)
    Htot = interTerm + fieldTerm + costTerm
end

Htot (generic function with 1 method)

## Aplicación del algoritmo MMC